In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##1. 파일 읽기
> tmdb_5000_movies.csv 파일을 읽어 크기를 확인하세요

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec4_BigData/5.컨텐츠기반/data/tmdb_5000_movies.csv')
df.shape

(4803, 20)

> 4803개의 영화와 20개의 feature가 존재하는 것을 알 수 있다.

##2. feature 줄이기

In [4]:
df.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


> 위의 feaure중 다음과 같은 내용만 추출해보자.
* id : 영화 아이디
* title : 영화 제목
* genres : 장르
* vote_average : 평점
* vote_count : 투표수
* popularity : 인기도
* keyword : 주요 키워드
* overview : 영화 개요

In [5]:
df_movies = df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]

df_movies.shape

(4803, 8)

#3. 데이터 클린징
> 위 데이터를 이용하여 이상치 및 결측치를 확인하세요

In [6]:
df_movies.isna().sum()

id              0
title           0
genres          0
vote_average    0
vote_count      0
popularity      0
keywords        0
overview        3
dtype: int64

In [7]:
df_movies[df_movies['overview'].isna()]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
2656,370980,Chiamatemi Francesco - Il Papa della gente,"[{""id"": 18, ""name"": ""Drama""}]",7.3,12,0.738646,"[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...",NaN
4140,459488,"To Be Frank, Sinatra at 100","[{""id"": 99, ""name"": ""Documentary""}]",0.0,0,0.050625,"[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...",NaN
4431,292539,Food Chains,"[{""id"": 99, ""name"": ""Documentary""}]",7.4,8,0.795698,[],NaN


> 영화의 요약 줄거리인 overview에 nan값이 3개가 존재한다.
다만 너무 요약이라 삭제해도 상관없지 않나 이런생각이 듭니다.

> overview에 3개의 결측치가 존재하는 것을 확인했다. 우리가 분석할 내용에서 overview는 별도로 분석하지 않을 것이므로 무시할 것이다.

In [8]:
df_movies.describe()

,id,vote_average,vote_count,popularity
count,4803.000000,4803.000000,4803.000000,4803.000000
mean,57165.484281,6.092172,690.217989,21.492301
std,88694.614033,1.194612,1234.585891,31.816650
min,5.000000,0.000000,0.000000,0.000000
25%,9014.500000,5.600000,54.000000,4.668070
50%,14629.000000,6.200000,235.000000,12.921594
75%,58610.500000,6.800000,737.000000,28.313505
max,459488.000000,10.000000,13752.000000,875.581305


In [9]:
def getLowUpperFences(data_frame, col, lowstate, upperstate):
  q1 = data_frame.describe().loc['25%'][col]
  q3 = data_frame.describe().loc['75%'][col]
  IQR = q3 - q1

  Lowerfence = q1 - (IQR * lowstate)
  Upperfence = q3 + (IQR * upperstate)

  return round(Lowerfence,2), round(Upperfence, 2)

In [10]:
lower, upper = getLowUpperFences(df_movies, 'vote_average', 1.5, 1.5)
df_movies[(df_movies['vote_average'] < lower) | (df_movies['vote_average'] > upper)].sort_values('vote_average', ascending=False)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
4247,361505,Me You and Five Bucks,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 35, ...",10.0,2,0.094105,[],"A womanizing yet lovable loser, Charlie, a wai..."
3519,89861,Stiff Upper Lips,"[{""id"": 35, ""name"": ""Comedy""}]",10.0,1,0.356495,"[{""id"": 131, ""name"": ""italy""}, {""id"": 8250, ""n...",Stiff Upper Lips is a broad parody of British ...
4045,78373,"Dancer, Texas Pop. 81","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",10.0,1,0.376662,"[{""id"": 1415, ""name"": ""small town""}, {""id"": 15...","Four guys, best friends, have grown up togethe..."
4662,40963,Little Big Top,"[{""id"": 35, ""name"": ""Comedy""}]",10.0,1,0.092100,"[{""id"": 10183, ""name"": ""independent film""}]",An aging out of work clown returns to his smal...
3992,346081,Sardaarji,[],9.5,2,0.296981,[],A ghost hunter uses bottles to capture trouble...
...,...,...,...,...,...,...,...,...
4458,279759,Harrison Montgomery,[],0.0,0,0.006943,[],Film from Daniel Davila
4444,146882,Elza,"[{""id"": 18, ""name"": ""Drama""}]",0.0,0,0.007254,"[{""id"": 254, ""name"": ""france""}, {""id"": 2071, ""...",A young Parisian woman of Caribbean descent re...
4400,219716,Sparkler,[],0.0,1,0.547654,[],Melba is a Californian trailer-park girl who i...
4309,355629,The Blade of Don Juan,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...",0.0,0,0.021678,"[{""id"": 5967, ""name"": ""peru""}]",The fate of an arm-wrestling match leads two r...


#4. 데이터 확인(팁)

In [11]:
df_movies[['genres', 'keywords']][:3]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."


> 전체 데이터가 화면에 다 표시되지 않는다.

In [12]:
pd.set_option('max_colwidth', 100)
df_movies[['genres', 'keywords']][:3]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic is..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret..."


> pandas의 set_option을 이용하여 columns의 최대 길이를 증가 시킬 수 있다.

In [13]:
pd.reset_option('max_colwidth')
df_movies[['genres', 'keywords']][:3]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name..."


> reset_option을 이용하면 초기화 시킬 수 있다.

#5. 데이터 가공

In [14]:
str_dict = '{"id": 28, "name": "Action"}'
print(type(str_dict))
print(str_dict)

<class 'str'>
{"id": 28, "name": "Action"}


>type은 str이고, 데이터가 dict형식 처럼 보인다

In [15]:
#srt_dict는 str형으로 나오기 때문에 dict처럼 검색하면 에러가 난다.
srt_dict['id']

NameError: ignored

> 그러나 위 코드처럼 dict처럼 접근하면 에러가 발생한다. 이는 모양만 dict인 문자열이기 때문에접근이 불가능하게 되는 것이다.

In [16]:
from ast import literal_eval
dict_str = literal_eval(str_dict)

print(type(dict_str))
print(dict_str)
print(dict_str['id'])

<class 'dict'>
{'id': 28, 'name': 'Action'}
28


In [17]:
#df_movies['genres'][:1][0]
#df_movies['genres'] = df_movies['genres'].apply(lambda x : literal_eval(x))
df_movies['genres'][:1][0][0]
literal_eval(df_movies['genres'][:1][0])

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

> "df_movies['genres'][:1][0]"이 정보는 리스트 모양을 갖지만 실제 자료형은 str형식이다. 

In [18]:
df_movies['genres'] = df_movies['genres'].apply(literal_eval)
print(type(df_movies['genres']))
df_movies['genres'][:1][0][0]

<class 'pandas.core.series.Series'>


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{'id': 28, 'name': 'Action'}

> literal_eval 함수를 이용하여 STR형식을 list형식으로 변환시키고, df_movies['genres'][:1][0][0]을 이용하여 추출했더니 원하는 정보인 {'id': 28, 'name': 'Action'}가 제대로 출력되었다.

In [19]:
df_movies['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

> 장르 처음 행의 처음 데이터를 학인하면 총 4개의 장르 정보가 속해있는 것을 볼 수 있다.

In [20]:
df_movies['genres'] = df_movies['genres'].apply(lambda x : [y['name'] for y in x])
df_movies['genres'][2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


['Action', 'Adventure', 'Crime']

> apply를 이용하여 장르 부분에 기존 데이터 대신 name정보만 입력했다.

In [21]:
df_movies.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,overview
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[Action, Adventure, Crime]",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[Action, Adventure, Science Fiction]",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


#6. feature생성

In [22]:
'==='.join(df_movies['genres'][0])

'Action===Adventure===Fantasy===Science Fiction'

> join은 각 데이터에 구분자를 지정하는 함수이다. 위에서 0번째 위치의 데이터를 '==='의 식별자로 구분하여 결과를 출력했다.

In [23]:
df_movies['genres_literal'] = df_movies['genres'].apply(lambda x : (' ').join(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [24]:
df_movies['genres_literal'].head(2)

0    Action Adventure Fantasy Science Fiction
1                    Adventure Fantasy Action
Name: genres_literal, dtype: object

> CountVectorizer에서 사용하기 위해 데이터를 띄어쓰기 구분자로 저장 하였다.

In [101]:
from sklearn.feature_extraction.text import CountVectorizer

#ngram_range 옵션은 1~2단어의 조합까지 표현하겠다 라는 뜻이다.
count_vect = CountVectorizer(ngram_range=(1,2)).fit(df_movies['genres_literal'])
count_vect.vocabulary_

> CountVectorizer에서 ngram_range는 단어의 조합을 어떻게 처리할 것이냐를 정하는 것으로 다음과 같이 사용할 수 있다.
* ngram_range(min_n, max_n)
* min_n : 최소 단어 수
* max_n : 최대 단어 수 
* 최소 단어 수에서 최대 단어 수까지 조합으로 count할 단어를 생성

> 단어가 너무 많으면 처리하는데 많은 시간이 소요되어 2개의 단어조합 까지만 사용한다.

min_df는 최소 빈도 수를 의미하며 0을 사용하면 모든 단어들의 조합을 사용하겠다는 의미이다. 만약 100이 입력되면, 100번 이하로 나타나는 조합은 배제하겠다는 의미이다.

In [26]:
genre_matrix = count_vect.fit_transform(df_movies['genres_literal'])
genre_matrix.shape

(4803, 276)

#7. 유사도 분석

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

genre_similarity = cosine_similarity(genre_matrix, genre_matrix)

genre_similarity[:3]

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ]])

> 간단하게 유사도 측정을 했다. 총 4803개의 데이터가 존재하며 각 데이터들의 유사도로 나타나기 때문에 영화를 찾기에는 어렵다. 따라서 결과를 정렬된 index정보로 출력해 볼 것이다.

In [28]:
#[:, ::-1] 이건 내림차순이라는 뜻이다.
genre_similarity_sorted_idx = genre_similarity.argsort()[:, ::-1]
genre_similarity_sorted_idx[:3]

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401]])

> argsort()는 배열 정렬시 사용하는 함수로, 기본은 오름차순으로 정렬된다.  하지만 우리가 필요한 정보는 유사도가 높은 정보순으로 정렬해야 함으로 내림차순으로 정렬하기 위해 [:. ::-1]을 이용했다. 또한 출력 결과는 유사도가 아닌 index인 것에 유의하자.

> 위의 정보에서 0번 영화와 가장 유사한 영화는 3494->813 순인것을 알 수 있다.


In [29]:
print(df_movies.iloc[0]['title'], df_movies.iloc[3494]['title'], df_movies.iloc[813]['title'], sep='\n')
print(df_movies.iloc[0], df_movies.iloc[3494], df_movies.iloc[813], sep='\n\n')

Avatar
Beastmaster 2: Through the Portal of Time
Superman
id                                                            19995
title                                                        Avatar
genres                [Action, Adventure, Fantasy, Science Fiction]
vote_average                                                    7.2
vote_count                                                    11800
popularity                                                  150.438
keywords          [{"id": 1463, "name": "culture clash"}, {"id":...
overview          In the 22nd century, a paraplegic Marine is di...
genres_literal             Action Adventure Fantasy Science Fiction
Name: 0, dtype: object

id                                                            27549
title                     Beastmaster 2: Through the Portal of Time
genres                [Action, Adventure, Fantasy, Science Fiction]
vote_average                                                    4.6
vote_count                        

> iloc를 이용하여 index정보로 장르가 비슷한 영화를 찾아봤다. 
즉, 영화의 정보를 얻기 위해 iloc에 index정보를 넣고 그 중 title feature를 출력하여 추천 영화들을 얻을 수 있게 된다.

In [30]:
df_movies.iloc[0]['title']

'Avatar'

#8. 영화 추천하기

In [31]:
col_list = ['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']
movies = df.copy()
movies = movies[col_list]
movies.set_index('title', inplace=True)
movies.head()

,id,genres,vote_average,vote_count,popularity,keywords,overview
title,,,,,,,
Avatar,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
Pirates of the Caribbean: At World's End,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
Spectre,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
The Dark Knight Rises,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
John Carter,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."


In [32]:
def StrToList(data_frame, col):
  data_frame[col] = data_frame[col].apply(literal_eval)
  data_frame[col] = data_frame[col].apply(lambda x: [y['name'] for y in x])
  data_frame[col] = data_frame[col].apply(lambda x : (' ').join(x))
StrToList(movies, 'genres')
movies.head(3)

,id,genres,vote_average,vote_count,popularity,keywords,overview
title,,,,,,,
Avatar,19995,Action Adventure Fantasy Science Fiction,7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
Pirates of the Caribbean: At World's End,285,Adventure Fantasy Action,6.9,4500,139.082615,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
Spectre,206647,Action Adventure Crime,6.3,4466,107.376788,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...


#8-2. 강사님 코드

In [33]:
#df_movies['title'] == "Avatar"
df_movies[df_movies['title']=="Avatar"]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction


In [34]:
df_movies['title'].isin(['Avatar'])
df_movies[df_movies['title'].isin(['Avatar'])]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction


> 위와 같이 비교 연산을 이용해도 되고, isin함수를 이용하여 데이터를 얻을 수도 있다.

In [35]:
movie_name = 'avatar'
#movie_name.upper()
#movie_name.lower()
df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])]

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy Science Fiction


> 사용자가 입력하는 정보는 대문자, 소문자를 구분하지 않고 사용하기 때문에 하나로 통일해야 검색이 가능하다. upper함수는 모든 문자를 대문자로 변환하며 lower함수는 모든 문자를 소문자로 변환한다.

> 여기서는 사용자가 입력하는 문자를 소문자로 변환하기 위해 movie_name.upper()를 사용했다. 또한 기존에 영화 제목정보 들은 series형으로 되어있어 문자열로 변환하기 위해 str을 사용했으며 마찬가지로 lower함수를 통해 소문자로 통일하였다.

> 그러나 우리가 찾고싶은건 검색을 위해 필요한 그 영화의 인덱스 값이다. 따라서 인덱스 값을 반환하도록 하다.

In [36]:
movie_name = 'superman'
#df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])]
#df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])].index
#df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])].index.values
title_movie = df_movies[df_movies['title'].str.lower().isin([movie_name.lower()])]

> 찾고자 하는 영화의 제목을 입력하면 영화 정보를 얻었다. 하지만 우리가 하고 싶은 것은 입력한 영화에 대한 추천 영화를 받고 싶기 때문에 index정보가 필요하다. 그래서 검색한 영화 정보에서 index.values를 이용하여 찾고자 하는 영화의 index정보를 얻게 되었다.

In [37]:
genre_similarity_sorted_idx[:4, :10]

array([[   0, 3494,  813,  870,   46,   14, 1296, 1652,  419,  420],
       [ 262,    1,  129,  199,  206,  208,   98, 2343, 2390,  379],
       [   2, 1740, 1542, 1082,  873, 1615,  969, 1058, 1073, 1076],
       [2195, 1850, 3316, 2218, 2435, 3073, 1503, 1470, 4230,  629]])

> [:4, :10]은 행을 4개까지 열을 10개까지 출력하는 기능을 한다.

> 위의 결과를 보면 4개의 영화 정보에 대한 10개의 추천 영화 index를 반환한 결과이다.

In [38]:
genre_similarity_sorted_idx[0, :10]

array([   0, 3494,  813,  870,   46,   14, 1296, 1652,  419,  420])

> 위의 결과는 0번 영화에 대한 top10의 추천 영화 정보이다.

In [39]:
movie_index = title_movie.index.values[0]
movie_index

813

In [40]:
genre_similarity_sorted_idx[movie_index, :10]

array([   0, 3494,  813,  870,   46,   14, 1296, 1652,  419,  420])

> 사용자가 입력한 정보에 대한 상위 10개의 추천영화 index정보이다.

###8.1 Quiz
> 영화 이름을 검색하면 10개의 추천 영화 제목과 평점을 출력하도록 코딩하시오

In [41]:
def findSimilarMovies(movie_name):
  
  #데이터프레임에서 입력된 영화의 인덱스 얻기
  movie_index = df_movies[df_movies['title'].str.lower() == movie_name.lower()].index[0]

  #영화 인덱스로 similarity 리스트 검색 -> top10이 나옴
  similar_movie_index = genre_similarity_sorted_idx[movie_index, :10]
  #입력한 영화가 추천으로 나오지 않게 조건을 달아줌
  if (movie_index in similar_movie_index):
    similar_movie_index = genre_similarity_sorted_idx[movie_index, :11]
  #top10의 인덱스를 가지고 제목과, 평점을 출력해줌

  print("top10 index : ", similar_movie_index)
  print('Genre : ', df_movies.iloc[movie_index]['genres'])
  print()
  print("Similar Movies are...")

  for idx in similar_movie_index:
    if idx == movie_index:
      continue
    print("Movie title : {}\nVote Average : {}".format(df_movies.iloc[idx]['title'], df_movies.iloc[idx]['vote_average']), end='\n\n')


In [42]:
movie_name = input("Please insert movie name: ")
#movie_name = 'superman'

findSimilarMovies(movie_name)

Please insert movie name: avatar
top10 index :  [   0 3494  813  870   46   14 1296 1652  419  420 1191]
Genre :  ['Action', 'Adventure', 'Fantasy', 'Science Fiction']

Similar Movies are...
Movie title : Beastmaster 2: Through the Portal of Time
Vote Average : 4.6

Movie title : Superman
Vote Average : 6.9

Movie title : Superman II
Vote Average : 6.5

Movie title : X-Men: Days of Future Past
Vote Average : 7.5

Movie title : Man of Steel
Vote Average : 6.5

Movie title : Superman III
Vote Average : 5.3

Movie title : Dragonball Evolution
Vote Average : 2.9

Movie title : Jumper
Vote Average : 5.9

Movie title : Hellboy II: The Golden Army
Vote Average : 6.5

Movie title : Small Soldiers
Vote Average : 6.2



> 위와 같이 입력된 영화에 따른 top10의 영화를 추천해 줄 수 있다.

In [43]:
movie_list = genre_similarity_sorted_idx[0, :10]

In [44]:
print(movie_list.ndim)

1


> ndim은 배열의 차원을 나타내는 객체이다. 현재 movie_top10의 차원을 알아보기 위해 ndim을 사용하였다.

> reshape 함수는 차원을 변경하기 위한 함수로 -1을 입력하게 되면 그 차원은 자동으로 설정하게 되며 현재는 01만 입력되어 1차원으로 설정하게 된다. (난 왠지 처음부터 1차원이었다.)

#9. 가중치

In [45]:
df_movies[['title', 'vote_average', 'vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


> 위에서 보듯이 평점이 좋지만 투표수가 적은 경우의 영화를 추천하기에는 문제가 있다. 따라서 가중치를 이용한 추천 시스템을 구축할 것이다. 다음은 유명한 영화 평 점 사이트인 IMDB에서 사용한 공식이다.
* 가중 평점 = (v/(v+m)) * R + (m/(v+m)) * C
* v : 개별 영화에 평점을 투표한 횟수
* m : 평점을 부여하기 위한 최소 투표 횟수
* R : 개별 영화에 대한 평균 평점
* C : 전체 영화에 대한 평균 평점

In [46]:
#전체 영화에 대한 평균 평점
C = df_movies['vote_average'].mean()
C
#평점을 부여하기 위한 최소 투표 횟수
m = df_movies['vote_count'].quantile(0.6)
print('C : ', round(C, 3))
print('m : ', round(m, 3))

C :  6.092
m :  370.2


> 평균 평점을 구하기 위해 mean함수를 이용했다. 

> quantile 함수는 입력된 데이터의 기준 값을 얻는 함수 이다. quantile(0.6)은 60%지점의 값을 얻은 것이며 370회의 투표를 한 것임을 알 수 있다.

> 최소한 370표의 투표를 받은 작품은 되어야 인정한다.

In [47]:
# 개별 영화에 평점을 투표한 횟수
v = df_movies['vote_count']

#개별 영화에 대한 평균 평점
R = df_movies['vote_average']
R

0       7.2
1       6.9
2       6.3
3       7.6
4       6.1
       ... 
4798    6.6
4799    5.9
4800    7.0
4801    5.7
4802    6.3
Name: vote_average, Length: 4803, dtype: float64

In [48]:
(v/(v+m)) * R + (m/(v+m)) * C

0       7.166301
1       6.838594
2       6.284091
3       7.541095
4       6.098838
          ...   
4798    6.290894
4799    6.089611
4800    6.106650
4801    6.084894
4802    6.100782
Length: 4803, dtype: float64

> 공식을 이용하여 각 영화들의 가중치 평점을 구했다.

In [60]:
df_movies['weighted_vote'] = (v/(v+m)) * R + (m/(v+m)) * C
df_movies[['title', 'vote_average', 'vote_count', 'weighted_vote']].sort_values('vote_average', ascending=False)[:10]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,title,vote_average,vote_count,weighted_vote
3519,Stiff Upper Lips,10.0,1,6.102699
4247,Me You and Five Bucks,10.0,2,6.113170
4045,"Dancer, Texas Pop. 81",10.0,1,6.102699
4662,Little Big Top,10.0,1,6.102699
3992,Sardaarji,9.5,2,6.110483
2386,One Man's Hero,9.3,2,6.109409
2970,There Goes My Baby,8.5,2,6.105110
1881,The Shawshank Redemption,8.5,8205,8.396052
2796,The Prisoner of Zenda,8.4,11,6.158767
3337,The Godfather,8.4,5893,8.263591


> 실행 결과 투표수가 적은 영화는 가중치 평점이 낮았으며, 투표수가 많은 영화일 수록 실 평점과 비슷한 수준으로 가중치 평점이 나타났다. 이를 이용하여 영화를 추천해 보자.

#9.1 가중치 평균을 이용하여 상위 10개의 영화를 추천하는 프로그램을 만드시오

>1번

In [52]:
df_movies[['title', 'vote_average', 'vote_count', 'weighted_vote']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,vote_count,weighted_vote
1881,The Shawshank Redemption,8.5,8205,8.396052
3337,The Godfather,8.4,5893,8.263591
662,Fight Club,8.3,9413,8.216455
3232,Pulp Fiction,8.3,8428,8.207102
65,The Dark Knight,8.2,12002,8.136930
1818,Schindler's List,8.3,4329,8.126069
3865,Whiplash,8.3,4254,8.123248
809,Forrest Gump,8.2,7927,8.105954
2294,Spirited Away,8.3,3840,8.105867
2731,The Godfather: Part II,8.3,3338,8.079586


>2번

In [99]:
def getSimilarMovies(movie_name):
  movie_index = df_movies[df_movies['title'].str.lower() == movie_name.lower()].index[0]
  movie_list = genre_similarity_sorted_idx[movie_index, :20]

  if movie_index in movie_list:
    movie_list = genre_similarity_sorted_idx[movie_index, :21]
    movie_list = movie_list[movie_list != movie_index]
  return df_movies.iloc[movie_list][['title', 'vote_average', 'vote_count','weighted_vote']].sort_values('weighted_vote', ascending=False).head(10)
#  movie_top10 = df_movies.iloc[movie_list]

In [100]:
#movie_name = input("Please insert movie name: ")
movie_name = 'superman'

movie_top10_df = getSimilarMovies(movie_name)
movie_top10_df
#df_movies.iloc[sim_movie_list]
  


,title,vote_average,vote_count,weighted_vote
46,X-Men: Days of Future Past,7.5,6032,7.418594
0,Avatar,7.2,11800,7.166301
207,Total Recall,7.1,1710,6.920643
14,Man of Steel,6.5,6359,6.477564
420,Hellboy II: The Golden Army,6.5,1527,6.420421
870,Superman II,6.5,629,6.348901
232,The Wolverine,6.3,4053,6.282606
3208,Star Wars: Clone Wars: Volume 1,8.0,27,6.221858
1191,Small Soldiers,6.2,511,6.154700
1932,Sheena,5.0,22,6.030907
